In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sqlite3

# Import TMDB Data

In [26]:
tmdb_movies = pd.read_csv('../data/tmdb.movies.csv.gz')
tmdb_movies.rename(columns={'title': 'primary_title'}, inplace=True)
tmdb_movies.drop(columns=['genre_ids', 'original_language', 'original_title'], inplace=True)
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     26517 non-null  int64  
 1   id             26517 non-null  int64  
 2   popularity     26517 non-null  float64
 3   release_date   26517 non-null  object 
 4   primary_title  26517 non-null  object 
 5   vote_average   26517 non-null  float64
 6   vote_count     26517 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 1.4+ MB


# The Numbers Data

In [27]:
tn_movie_budgets = pd.read_csv('../data/tn.movie_budgets.csv.gz')
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [28]:
tn_movie_budgets['release_date'] = pd.to_datetime(tn_movie_budgets['release_date'])
tn_movie_budgets['release_month'] = pd.to_datetime(tn_movie_budgets['release_date']).dt.month_name()
tn_movie_budgets['release_year'] = pd.to_datetime(tn_movie_budgets['release_date']).dt.year

tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].str.replace('$', '')
tn_movie_budgets['domestic_gross'] = tn_movie_budgets['domestic_gross'].str.replace(',', '').astype(int) 

tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].str.replace('$', '')
tn_movie_budgets['worldwide_gross'] = tn_movie_budgets['worldwide_gross'].str.replace(',', '').astype(int)

tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].str.replace('$', '')
tn_movie_budgets['production_budget'] = tn_movie_budgets['production_budget'].str.replace(',', '').astype(int)

tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_month      5782 non-null   object        
 7   release_year       5782 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(4), object(2)
memory usage: 338.9+ KB


In [29]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
tn_movie_budgets['release_month'] = pd.Categorical(tn_movie_budgets['release_month'], categories=months, ordered=True)
tn_movie_budgets['release_month'].value_counts()

release_month
December     745
October      573
August       496
September    493
November     486
June         479
March        470
April        454
July         440
May          407
February     392
January      347
Name: count, dtype: int64

In [52]:
tn_movie_budgets.rename(columns={'movie': 'primary_title'}, inplace=True)
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   primary_title      5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   release_month      5782 non-null   category      
 7   release_year       5782 non-null   int32         
dtypes: category(1), datetime64[ns](1), int32(1), int64(4), object(1)
memory usage: 299.8+ KB


# Removing zeros


In [30]:
tn_movie_budgets_nonzero = tn_movie_budgets[tn_movie_budgets['domestic_gross'] != 0].copy()

# SQL DB

In [31]:
conn = sqlite3.connect('../data/im.db')
pd.read_sql('SELECT name AS "Table Names" FROM sqlite_master WHERE type="table"', conn)

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [32]:
# Making the IMDB movie_basics table into a pandas df
imdb_movie_basics = pd.read_sql('SELECT * FROM movie_basics', conn)
imdb_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [33]:
# Making the IMDB directors table into a pandas df
imdb_directors = pd.read_sql('SELECT * FROM directors', conn)
imdb_directors.value_counts()

movie_id   person_id
tt2249786  nm3911770    52
           nm1286055    52
           nm2853432    52
           nm2747918    52
           nm2583932    52
                        ..
tt1997402  nm0950841     1
tt1997405  nm1016009     1
tt1997417  nm0133980     1
tt1997429  nm4554137     1
tt4105968  nm1297226     1
Name: count, Length: 163535, dtype: int64

In [39]:
imdb_movie_akas = pd.read_sql('SELECT * FROM movie_akas', conn)
imdb_movie_akas.drop(columns=['is_original_title'], inplace=True)
imdb_movie_akas.head()

,movie_id,ordering,title,region,language,types,attributes
0,tt0369610,10,Джурасик свят,BG,bg,None,None
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None


In [35]:
imdb_writers = pd.read_sql('SELECT * FROM writers', conn)
imdb_writers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255873 entries, 0 to 255872
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   255873 non-null  object
 1   person_id  255873 non-null  object
dtypes: object(2)
memory usage: 3.9+ MB


In [44]:
imdb_writers_and_directors2 = pd.read_sql("""
            SELECT *, 
            FROM movie_ratings
            INNER JOIN directors
                USING(movie_id)
            INNER JOIN writers
                USING(movie_id)

            
            """, conn)

In [50]:
d_e = pd.read_sql("""
SELECT m.averagerating, person_id, b.primary_title
FROM movie_ratings as m
    JOIN (SELECT DISTINCT movie_id, person_id
FROM directors
GROUP BY person_id
HAVING COUNT() >5) as d_n_e
    ON d_n_e.movie_id = m.movie_id
    JOIN movie_basics AS b
        USING(movie_id)
""", conn)
d_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7081 entries, 0 to 7080
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   averagerating  7081 non-null   float64
 1   person_id      7081 non-null   object 
 2   primary_title  7081 non-null   object 
dtypes: float64(1), object(2)
memory usage: 166.1+ KB


In [43]:
# using left join
imdb_writers_and_directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46179231 entries, 0 to 46179230
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   movie_id       object 
 1   averagerating  float64
 2   numvotes       int64  
 3   person_id      object 
 4   person_id      object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.7+ GB


In [46]:
# using inner join
imdb_writers_and_directors2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46166010 entries, 0 to 46166009
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   movie_id       object 
 1   averagerating  float64
 2   numvotes       int64  
 3   person_id      object 
 4   person_id      object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.7+ GB


In [ ]:
pd.read_sql("""
            SELECT 
            FROM movie_basics AS m
            JOIN directors AS d
                USING(movie_id)
            JOIN writers AS w
                USING(movie_id)

            
            """, conn)

# Combining Dataframes


In [36]:
# Combing the IMDB 'movie_basics' table with the TMDB 'tmdb_movies' table

merged_movie_basics = pd.merge(imdb_movie_basics, tmdb_movies, on='primary_title')
merged_movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23612 entries, 0 to 23611
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         23612 non-null  object 
 1   primary_title    23612 non-null  object 
 2   original_title   23607 non-null  object 
 3   start_year       23612 non-null  int64  
 4   runtime_minutes  21750 non-null  float64
 5   genres           23263 non-null  object 
 6   Unnamed: 0       23612 non-null  int64  
 7   id               23612 non-null  int64  
 8   popularity       23612 non-null  float64
 9   release_date     23612 non-null  object 
 10  vote_average     23612 non-null  float64
 11  vote_count       23612 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 2.2+ MB


In [54]:
merged_exp_directors = pd.merge(d_e, tmdb_movies, on='primary_title')
merged_exp_directors = pd.merge(merged_exp_directors, tn_movie_budgets, on='primary_title')
merged_exp_directors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475 entries, 0 to 474
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   averagerating      475 non-null    float64       
 1   person_id          475 non-null    object        
 2   primary_title      475 non-null    object        
 3   Unnamed: 0         475 non-null    int64         
 4   id_x               475 non-null    int64         
 5   popularity         475 non-null    float64       
 6   release_date_x     475 non-null    object        
 7   vote_average       475 non-null    float64       
 8   vote_count         475 non-null    int64         
 9   id_y               475 non-null    int64         
 10  release_date_y     475 non-null    datetime64[ns]
 11  production_budget  475 non-null    int64         
 12  domestic_gross     475 non-null    int64         
 13  worldwide_gross    475 non-null    int64         
 14  release_mo